<a href="https://colab.research.google.com/github/p4zaa/OTTO-Multi-Objective-Recommender-System/blob/main/%5BDEVELOP1_6%5D_OTTO_%E2%80%93_Multi_Objective_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version Logs
* [View in my Github](https://github.com/p4zaa/OTTO-Multi-Objective-Recommender-System)

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

     |████████████████████████████████| 9.4 MB 2.6 MB/s 
     |████████████████████████████████| 4.5 MB 2.7 MB/s 
     |████████████████████████████████| 3.2 MB 2.6 MB/s 
     |████████████████████████████████| 873 kB 2.6 MB/s 
     |████████████████████████████████| 564 kB 5.5 MB/s 
     |████████████████████████████████| 280 kB 44.1 MB/s 


In [ ]:
import pandas as pd, numpy as np
from itertools import product
import io, os, json
import tqdm

import time

from sklearn.metrics import mean_squared_error

import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
pio.templates.default = "plotly_white"

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import to_hetero
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear, GraphConv
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import HGTLoader, NeighborLoader, LinkNeighborLoader

!pip install -q sentence-transformers

%matplotlib inline

     |████████████████████████████████| 85 kB 2.8 MB/s 
     |████████████████████████████████| 5.8 MB 9.8 MB/s 
     |████████████████████████████████| 1.3 MB 45.9 MB/s 
     |████████████████████████████████| 182 kB 64.9 MB/s 
     |████████████████████████████████| 7.6 MB 52.8 MB/s 


In [ ]:
!pip3 install -q torch torchvision torchaudio
!pip install -U polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 5.3 MB/s 


In [ ]:
import polars as pl

In [ ]:
print('Pytorch CUDA Version is ', torch.version.cuda)

Pytorch CUDA Version is  11.6


In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
torch.cuda.is_available()

False

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Competition Dataset

In [ ]:

from google.colab import files
files.upload() # expire any previous token(s) and upload recreated token
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets list


Saving kaggle.json to kaggle.json
rm: cannot remove '/root/.kaggle': No such file or directory
ref                                                           title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                     COVID-19 Dataset                                     5MB  2022-11-13 15:47:17          14954        424  1.0              
devrimtuner/list-of-moststreamed-songs-on-spotify             Top 100 Spotify Songs👑🎤🎧🎼                            3KB  2022-12-30 05:42:54            332         26  1.0              
thedevastator/analyzing-credit-card-spending-habits-in-india  Credit Card Spending Habits in India               319KB  2022-12-14 07:30:37           1522         52

In [ ]:
!kaggle datasets download 'radek1/otto-full-optimized-memory-footprint' -p /content/kaggle/ --unzip

 99% 1.07G/1.09G [00:05<00:00, 195MB/s]
100% 1.09G/1.09G [00:05<00:00, 197MB/s]


## Files
- **train.jsonl** - the training data, which contains full session data
  * `session` - the unique session id
  * `events` - the time ordered sequence of events in the session
    * `aid` - the article id (product code) of the associated event
    * `ts` - the Unix timestamp of the event
    * `type` - the event type, i.e., whether a product was clicked, added to the user's cart, or ordered during the session
###### {'clicks': 0, 'carts': 1, 'orders': 2}
- **test.jsonl** - the test data, which contains truncated session data
  * your task is to predict the next `aid` clicked after the session truncation, as well as the the remaining `aids` that are added to `carts` and `orders`; you may predict up to 20 values for each session `type`
- **sample_submission.csv** - a sample submission file in the correct format

In [ ]:
# Pandas
#df = pd.read_parquet('/content/kaggle/train.parquet')

# Polars
df = pl.read_parquet('/content/kaggle/train.parquet')

In [ ]:
# sample 10%(frac=0.1) of data
#df = df.sample(frac=0.01, replace=False)

# Polars
df = df.sample(frac=0.005)

In [ ]:
df = df.to_pandas()

In [ ]:
df

,session,aid,ts,type
0,1501601,635864,1659443915,0
1,644159,396198,1660569107,0
2,7936838,1116026,1660412341,0
3,7560153,1672263,1660337516,0
4,3424360,1076444,1660928522,0
...,...,...,...,...
1083575,553176,1259905,1660404960,0
1083576,387336,1449938,1661648656,0
1083577,9147729,491706,1660624426,0
1083578,1569592,1017009,1660042861,0


In [ ]:
#df.isna().sum()

In [ ]:
df['type'].unique()

array([0, 1, 2], dtype=uint8)

In [ ]:
#df.info()

# Construct Graph Data (memory-efficient optimized)

### `edge_index` new code construct

In [ ]:
# session index dict
session = df['session'].unique()
source_idx = {id:idx for idx, id in enumerate(session)}

# aid(article id) index dict
aid = df['aid'].unique()
target_idx = {id:idx for idx, id in enumerate(aid)}

In [ ]:
connected = df[['session', 'aid']]
connected['session'] = connected['session'].map(source_idx)
connected['aid'] = connected['aid'].map(target_idx)

<ipython-input-21-fdbbf2e7eb54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connected['session'] = connected['session'].map(source_idx)
<ipython-input-21-fdbbf2e7eb54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  connected['aid'] = connected['aid'].map(target_idx)


In [ ]:
source = connected['session']
target = connected['aid']
edge_index = torch.tensor((source.values, target.values))

<ipython-input-22-23961f25841f>:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  edge_index = torch.tensor((source.values, target.values))


### Nodes and Edges Attribute

In [ ]:
df['session']

0          1501601
1           644159
2          7936838
3          7560153
4          3424360
            ...   
1083575     553176
1083576     387336
1083577    9147729
1083578    1569592
1083579    6017748
Name: session, Length: 1083580, dtype: int32

In [ ]:
## Nodes Atrributes
session_num_nodes = df['session'].nunique()
aid_num_nodes = df['aid'].nunique()
aid_features = torch.rand((aid_num_nodes, 32)) # Create (random) article features with shape [num_node_aid, dimensions]

## Edges Atrributes
edge_index = edge_index
edge_label = torch.tensor(df['type'].values).type(torch.int64)

In [ ]:
del df

### Construct HeteroData

In [ ]:
node_types = {
    'session': {
        #'num_nodes': session_num_nodes,
        'node_id': torch.tensor(list(source_idx.values()), dtype=torch.int64)
    },
    'aid': {
        'x': aid_features,
        'node_id': torch.tensor(list(target_idx.values()), dtype=torch.int64)
    }
}

edge_types = {
    ('session', 'event', 'aid'): {
        'edge_index': edge_index,
        'edge_label': edge_label
    }#,
    #('session', 'cart', 'aid'): {
        
    #},
    #('session', 'buy', 'aid'): {
        
    #}
}

In [ ]:
data = HeteroData({**node_types, **edge_types})

In [ ]:
data

HeteroData(
  session={ node_id=[912038] },
  aid={
    x=[380386, 32],
    node_id=[380386]
  },
  (session, event, aid)={
    edge_index=[2, 1083580],
    edge_label=[1083580]
  }
)

In [ ]:
data.metadata()

(['session', 'aid'], [('session', 'event', 'aid')])

In [ ]:
node_types, edge_types = data.metadata()
print('Node types:', node_types)
print('Edge types:',edge_types)

Node types: ['session', 'aid']
Edge types: [('session', 'event', 'aid')]


In [ ]:
#print('Isolated nodes?', data.has_isolated_nodes())
#print('Self loops?', data.has_self_loops())
#print('Undirected graph? ', data.is_undirected())

# Graph-based Modeling [follow [this](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py) sample]

In [ ]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Construct Undirected Graph

In [ ]:
# add sesion features for message passing:
data['session'].x = torch.rand(data['session'].num_nodes, 32)

In [ ]:
# Add user node features for message passing:
#data['session'].x = torch.eye(data['session'].num_nodes, device=device)
#del data['session'].num_nodes

In [ ]:
# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)

In [ ]:
del data['aid', 'rev_event', 'session'].edge_label  # Remove "reverse" label.

In [ ]:
data

HeteroData(
  session={
    node_id=[912038],
    x=[912038, 32]
  },
  aid={
    x=[380386, 32],
    node_id=[380386]
  },
  (session, event, aid)={
    edge_index=[2, 1083580],
    edge_label=[1083580]
  },
  (aid, rev_event, session)={ edge_index=[2, 1083580] }
)

### Calculate Weight

In [ ]:
data['session', 'aid'].edge_label.shape

torch.Size([1083580])

In [ ]:
# We have an unbalanced dataset with many labels for rating 3 and 4, and very
# few for 0 and 1. Therefore we use a weighted MSE loss.

counts = torch.bincount(data['session', 'aid'].edge_label)

# Set weights normalized by (max count/each count)
weight = counts.max() / counts

In [ ]:
counts

tensor([973843,  84241,  25496])

In [ ]:
weight

tensor([ 1.0000, 11.5602, 38.1959])

In [ ]:
data_dict = {'type': (counts, '# rows','coral'), 'weights': (weight, 'weights','royalblue')}

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(x=np.arange(6), y=counts.detach().cpu().numpy(),
               name = 'nb rows', line_color= 'coral'))
fig.add_trace(
    go.Scatter(x=np.arange(6), y=weight.detach().cpu().numpy(),
               name = 'weights', line_color= 'royalblue'),  secondary_y=True)


fig.update_yaxes(title_text="# rows", secondary_y=False)
fig.update_yaxes(title_text="weights", secondary_y=True)
fig.update_xaxes(title_text="Type")
fig

### [Follow [this sample](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py)] Train/Val/Test Link Level Splits

In [ ]:
# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('session', 'event', 'aid')],
    rev_edge_types=[('aid', 'rev_event', 'session')],
)(data)

### Mini Batch

In [ ]:
# Define seed edges:
edge_label_index = train_data['session', 'event', 'aid'].edge_label_index
edge_label = train_data['session', 'event', 'aid'].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,  # TODO
    num_neighbors=[5, 10],  # TODO
    neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=64,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

#assert sampled_data["session", "event", "aid"].edge_label_index.size(1) == 128
#assert sampled_data["session", "event", "aid"].edge_label.min() == 0
#assert sampled_data["session", "event", "aid"].edge_label.max() == 2

Sampled mini-batch:
HeteroData(
  session={
    node_id=[320],
    x=[320, 32]
  },
  aid={
    x=[139, 32],
    node_id=[139]
  },
  (session, event, aid)={
    edge_index=[2, 321],
    edge_label=[64],
    edge_label_index=[2, 64],
    input_id=[64]
  },
  (aid, rev_event, session)={ edge_index=[2, 307] }
)


### [New weight calculation code]

In [ ]:
# We have an unbalanced dataset with many labels for rating 3 and 4, and very
# few for 0 and 1. Therefore we use a weighted MSE loss.
weight = torch.bincount(train_data['session', 'aid'].edge_label)
weight = weight.max() / weight

In [ ]:
weight

tensor([ 1.0000, 11.5589, 38.0843])

### Model and loss functions

#### Loss function

In [ ]:
def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

#### Model

In [ ]:
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)

        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # encoder and decoder
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

        # embedding matrices for sessions and aids
        self.aid_lin = Linear(-1, hidden_channels)
        self.session_emb = torch.nn.Embedding(data['session'].num_nodes, hidden_channels)
        self.aid_emb = torch.nn.Embedding(data['aid'].num_nodes, hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        x_dict = {
            'session': self.session_emb(data['session'].node_id.to(device)),
            'aid': self.aid_lin(data['aid'].x.to(device).float()) + self.aid_emb(data['aid'].node_id.to(device))
        }
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

In [ ]:
model = Model(hidden_channels=32).to(device)

In [ ]:
data

HeteroData(
  session={
    node_id=[912038],
    x=[912038, 32]
  },
  aid={
    x=[380386, 32],
    node_id=[380386]
  },
  (session, event, aid)={
    edge_index=[2, 1083580],
    edge_label=[1083580]
  },
  (aid, rev_event, session)={ edge_index=[2, 1083580] }
)

In [ ]:
# Due to lazy initialization, we need to run one model step so the number
# of parameters can be inferred:
'''
sampled = next(iter(train_loader))

with torch.no_grad():
    model.encoder(sampled_data.to(device).x_dict, sampled_data.to(device).edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
'''

'\nsampled = next(iter(train_loader))\n\nwith torch.no_grad():\n    model.encoder(sampled_data.to(device).x_dict, sampled_data.to(device).edge_index_dict)\n\noptimizer = torch.optim.Adam(model.parameters(), lr=0.01)\n'

In [ ]:
'''
def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['session', 'aid'].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)
'''

"\ndef train():\n    model.train()\n    optimizer.zero_grad()\n    pred = model(train_data.x_dict, train_data.edge_index_dict,\n                 train_data['session', 'aid'].edge_label_index)\n    target = train_data['session', 'aid'].edge_label\n    loss = weighted_mse_loss(pred, target, weight)\n    loss.backward()\n    optimizer.step()\n    return float(loss)\n"

In [ ]:
'''
@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=-1, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)
'''

"\n@torch.no_grad()\ndef test(data):\n    model.eval()\n    pred = model(data.x_dict, data.edge_index_dict,\n                 data['session', 'aid'].edge_label_index)\n    pred = pred.clamp(min=-1, max=2)\n    target = data['session', 'aid'].edge_label.float()\n    rmse = F.mse_loss(pred, target).sqrt()\n    return float(rmse)\n"

In [ ]:
def train(train_data=sampled_data):
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['session', 'aid'].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight.to(device))
    #loss = F.binary_cross_entropy_with_logits(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss), pred, model

In [ ]:
@torch.no_grad()
def test(data=data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse), pred, target

### [non-batch] Training Heterogenous Link-level GNN

In [ ]:
'''
for epoch in range(0, 30):
    loss = train()
    train_rmse = test(train_data)
    val_rmse = test(val_data)
    test_rmse = test(test_data)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '
          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')
'''

"\nfor epoch in range(0, 30):\n    loss = train()\n    train_rmse = test(train_data)\n    val_rmse = test(val_data)\n    test_rmse = test(test_data)\n    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_rmse:.4f}, '\n          f'Val: {val_rmse:.4f}, Test: {test_rmse:.4f}')\n"

### [mini-batch] Training Heterogenous Link-level GNN

In [ ]:
import torch.nn.functional as F

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

#model = Model(hidden_channels=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(0, 10):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        loss, pred, model = train(train_data=sampled_data.to(device))
        # TODO: Move `sampled_data` to the respective `device`
        # TODO: Run `forward` pass of the model
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`

        #loss.backward()
        #optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")

Device: 'cpu'


 26%|██▌       | 3476/13545 [2:47:06<8:04:03,  2.88s/it]


KeyboardInterrupt: ignored

In [ ]:
path = '/content/drive/MyDrive/OTTO-Kaggle-Competition/model.pt'
# Save the entire model
#torch.save(model, path)
#model

In [ ]:
# Load the entire model
trained_model = torch.load(path, map_location=torch.device('cpu'))
trained_model

Model(
  (encoder): GraphModule(
    (conv1): Module(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
    (conv2): Module(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
  )
  (decoder): EdgeDecoder(
    (lin1): Linear(64, 32, bias=True)
    (lin2): Linear(32, 1, bias=True)
  )
  (aid_lin): Linear(32, 32, bias=True)
  (session_emb): Embedding(208340, 32)
  (aid_emb): Embedding(127651, 32)
)

In [ ]:
data.x_dict

{'session': tensor([[0.3496, 0.3576, 0.4350,  ..., 0.6999, 0.4863, 0.4950],
         [0.1491, 0.8726, 0.9180,  ..., 0.6280, 0.2909, 0.3206],
         [0.0133, 0.7595, 0.0310,  ..., 0.4020, 0.3845, 0.7873],
         ...,
         [0.4796, 0.9141, 0.4633,  ..., 0.3266, 0.3514, 0.0129],
         [0.2017, 0.2295, 0.6426,  ..., 0.0386, 0.0270, 0.8272],
         [0.1999, 0.2325, 0.2433,  ..., 0.5451, 0.6901, 0.3934]]),
 'aid': tensor([[0.6101, 0.5663, 0.1656,  ..., 0.5417, 0.3627, 0.3910],
         [0.5994, 0.7613, 0.2289,  ..., 0.9359, 0.9888, 0.1481],
         [0.4539, 0.9263, 0.4720,  ..., 0.4157, 0.2312, 0.0429],
         ...,
         [0.5931, 0.5741, 0.2546,  ..., 0.9014, 0.7131, 0.1320],
         [0.8739, 0.8120, 0.1230,  ..., 0.0576, 0.0523, 0.0178],
         [0.5163, 0.5471, 0.3522,  ..., 0.8437, 0.4139, 0.9526]])}

In [ ]:
model.encoder(data.to(device).x_dict, data.to(device).edge_index_dict)

{'session': tensor([[ 0.5086,  0.1713, -0.1024,  ..., -0.2654,  0.0533,  0.1688],
         [ 0.5146, -0.0319, -0.0986,  ..., -0.2964, -0.2364,  0.0285],
         [ 0.8892,  0.2763, -0.2755,  ..., -0.3909,  0.0060,  0.1843],
         ...,
         [ 0.7275,  0.4005, -0.3474,  ..., -0.2208,  0.0366,  0.0864],
         [ 0.7869,  0.1656, -0.2175,  ..., -0.3619,  0.1895,  0.2474],
         [ 0.4761,  0.2052, -0.1818,  ..., -0.2033,  0.0482, -0.0524]],
        grad_fn=<AddBackward0>),
 'aid': tensor([[-0.0559, -0.2701,  0.3355,  ...,  0.4512, -0.2145,  0.3344],
         [-0.3173, -0.3699,  0.2872,  ...,  0.3806, -0.0325,  0.2324],
         [-0.3945, -0.1062,  0.3149,  ...,  0.3986, -0.2398,  0.4475],
         ...,
         [-0.1265,  0.0740,  0.4994,  ...,  0.3266, -0.2041,  0.7710],
         [-0.3580, -0.0395,  0.1333,  ...,  0.3520, -0.3750,  0.6459],
         [-0.5111, -0.3922,  0.0240,  ...,  0.4656, -0.0313,  0.6253]],
        grad_fn=<AddBackward0>)}

In [ ]:
trained_model.encoder(data.to(device).x_dict, data.to(device).edge_index_dict)

{'session': tensor([[-3.4276e-02, -6.7186e-02,  7.0359e-02,  ...,  5.6479e-01,
           6.2746e-01, -3.6857e-01],
         [ 9.0695e-03, -1.8986e-01, -4.2547e-02,  ...,  5.7287e-01,
           6.2704e-01,  1.6782e-01],
         [-1.3472e-01, -2.4179e-01,  1.1364e-01,  ...,  4.5827e-01,
           3.5611e-01,  2.0237e-01],
         ...,
         [-6.2032e-02,  1.2672e-01,  1.0672e-01,  ...,  3.6610e-01,
           3.7137e-01, -1.8489e-01],
         [ 1.9452e-04, -4.1146e-02, -1.7734e-02,  ...,  3.1574e-01,
           3.3442e-01, -1.9015e-01],
         [ 3.8809e-02, -3.9735e-03,  1.3751e-01,  ...,  4.9234e-01,
           3.3738e-01, -2.6441e-01]], grad_fn=<AddBackward0>),
 'aid': tensor([[-0.8280,  0.0782,  0.6897,  ...,  0.3859, -0.2916, -0.7016],
         [-0.6276,  0.2062,  0.6144,  ...,  0.3266, -0.3236, -0.5227],
         [-0.6649, -0.0069,  0.4999,  ...,  0.1297, -0.1545, -0.2722],
         ...,
         [-1.1918,  0.0282,  1.0897,  ...,  0.5333, -0.2856, -0.8711],
         [-0.2

### Evaluate a Heterogenous Linklevel GNN

In [ ]:
# Define the validation seed edges:
edge_label_index = val_data['session', 'event', 'aid'].edge_label_index
edge_label = val_data['session', 'event', 'aid'].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,  # TODO
    num_neighbors=[20, 10],  # TODO
    #neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data['session', 'event', 'aid'].edge_label_index.size(1) == 3 * 128
assert sampled_data['session', 'event', 'aid'].edge_label.min() >= 0
assert sampled_data['session', 'event', 'aid'].edge_label.max() <= 2

Sampled mini-batch:
HeteroData(
  session={
    node_id=[3508],
    x=[3508, 32]
  },
  aid={
    x=[1209, 32],
    node_id=[1209]
  },
  (session, event, aid)={
    edge_index=[2, 3238],
    edge_label=[384],
    edge_label_index=[2, 384],
    input_id=[384]
  },
  (aid, rev_event, session)={ edge_index=[2, 3463] }
)


In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# You need the labels to binarize
labels = [0, 1, 2]

preds = []
ground_truths = []
for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        _, pred, ground_truth = test(data=sampled_data.to(device))
        preds.append(pred)
        ground_truths.append(ground_truth)
        # TODO: Collect predictions and ground-truths and write them into
        # `preds` and `ground_truths`.
        #raise NotImplementedError

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()

# Binarize `pred` with shape (n_samples, n_classes)
pred = label_binarize(pred.astype(int), classes=labels)

# Binarize `ground_truth` with shape (n_samples, n_classes)
ground_truth = label_binarize(ground_truth, classes=labels)

#rmse = _
auc = roc_auc_score(ground_truth, pred, multi_class='ovo', average='weighted')
print()
print(f"Validation AUC: {auc:.4f}")
#print('RMSE: ', rmse)

100%|██████████| 283/283 [04:20<00:00,  1.09it/s]


Validation AUC: 0.4976


# Test dataset

In [ ]:
#df = pd.read_parquet('/content/kaggle/test.parquet')

In [ ]:
#df = df.sample(frac=0.01, replace=False)

In [ ]:
#df

## Construct heterogenous graph for test

In [ ]:
'''
# session index dict
session = df['session'].unique()
source_idx = {id:idx for idx, id in enumerate(session)}

# aid(article id) index dict
aid = df['aid'].unique()
target_idx = {id:idx for idx, id in enumerate(aid)}
'''

In [ ]:
'''
connected = df[['session', 'aid']]
connected['session'] = connected['session'].map(source_idx)
connected['aid'] = connected['aid'].map(target_idx)

source = connected['session']
target = connected['aid']
edge_index = torch.tensor((source.values, target.values)).type(torch.int64)
'''

In [ ]:
'''
## Nodes Atrributes
session_num_nodes = df['session'].nunique()
aid_num_nodes = df['aid'].nunique()
aid_features = torch.rand((aid_num_nodes, 300)) # Create (random) article features with shape [num_node_aid, dimensions]

## Edges Atrributes
edge_index = edge_index
edge_label = torch.tensor(df['type'].values).type(torch.int64)
'''

In [ ]:
'''
node_types = {
    'session': {
        'num_nodes': session_num_nodes
    },
    'aid': {
        'x': aid_features
    }
}

edge_types = {
    ('session', 'event', 'aid'): {
        'edge_index': edge_index,
        'edge_label': edge_label,
        'edge_label_index': edge_index
    }#,
    #('session', 'cart', 'aid'): {
        
    #},
    #('session', 'buy', 'aid'): {
        
    #}
}
'''

In [ ]:
#Rtest_data = HeteroData({**node_types, **edge_types})

In [ ]:
#data

In [ ]:
#Rtest_data

In [ ]:
'''
# add sesion features for message passing:
Rtest_data['session'].x = torch.rand(Rtest_data['session'].num_nodes, 300)

Rtest_data = T.ToUndirected()(Rtest_data)
del Rtest_data['aid', 'rev_event', 'session'].edge_label  # Remove "reverse" label.
'''

In [ ]:
#Rtest_data

## Testing

In [ ]:
'''
with torch.no_grad():
    model.eval()
    gg = model(Rtest_data.x_dict, Rtest_data.edge_index_dict, Rtest_data['session', 'aid'].edge_label_index)
'''

In [ ]:
#data['session', 'aid'].edge_label

In [ ]:
#Rtest_data['session', 'aid'].edge_label

In [ ]:
#gg

In [ ]:
#data

# [TEMPORARY DROP]

In [ ]:
# Temporary comment
'''
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, conv):
        super().__init__()
        # conv(#in_channels, #out_channels)
        ''''''
        in_channels (int or tuple): 
            Size of each input sample, or :obj:`-1` to
            derive the size from the first input(s) to the forward method.
            A tuple corresponds to the sizes of source and target
            dimensionalities.
        ''''''
        self.conv1 = conv((-1, -1), hidden_channels)
        self.conv2 = conv((-1, -1), out_channels)
        self.linear1 = Linear(-1, out_channels)
        self.linear2 = Linear(-1, out_channels)

    def forward(self, x, edge_index):
        x0 = self.linear1(x)
        x2 = self.conv1(x0, edge_index).relu()
        x3 = self.conv2(x2, edge_index)
        x4 = self.linear2(x2 + x3)
        # Add combined layer to reduce over-smoothing
        return x4

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)
        z = self.lin1(z).relu()
        z = self.lin2(z)
        return z.view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels,  conv=SAGEConv):
        super().__init__()
        self.encoder = GNNEncoder(hidden_channels, hidden_channels,  conv) # Initialize GNNEncoder
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels) # Initialze EdgeDecoder

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict) # Here the call from model.encoder(...)
        return self.decoder(z_dict, edge_label_index)
'''

In [ ]:
'''
def train(train_data, model, optimizer, loss=weighted_mse_loss):
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['session', 'aid'].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss.sqrt())
'''

In [ ]:
## set pred.clamp
'''
@torch.no_grad()
def test(data, model, metric=F.mse_loss):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse) # Return RMSE loss
'''

In [ ]:
#from tqdm import tqdm
#from IPython.display import clear_output

In [ ]:
"""
def train_test(model, model_params, learning_rate=0.01, e_patience = 10, min_acc= 0.05, n_epochs=500):
    t0 = time.time()

    model = model(**model_params) # Define the model

    # Due to lazy initialization, we need to run one model step so the number
    # of parameters can be inferred:
    with torch.no_grad():
        model.encoder(train_data.x_dict, train_data.edge_index_dict) # Run once with torch.no_grad() to get parameter for optimizer below

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    k=0
    loss, train_rmse, val_rmse, test_rmse = [], [], [], []
    train_wrmse, val_wrmse, test_wrmse = [], [], []
    for epoch in tqdm(range(n_epochs)):
        # Call train fuction here >> return loss
        loss += [train(train_data, model, optimizer, loss=weighted_mse_loss)]
        
        # Call test function here >> return RMSE loss
        train_wrmse += [test(train_data, model, metric=weighted_mse_loss)]
        train_rmse += [test(train_data, model, metric=F.mse_loss)]
        
        val_wrmse += [test(val_data, model, metric=weighted_mse_loss)]
        val_rmse += [test(val_data, model, metric=F.mse_loss)]
        
        test_wrmse += [test(test_data, model, metric=weighted_mse_loss)]
        test_rmse += [test(test_data, model, metric=F.mse_loss)]

        if epoch+1 %10==0:
            print(f'Epoch: {epoch+1:03d}, Loss: {loss[-1]:.4f}, Train: {train_rmse[-1]:.4f}, '
                  f'Val: {val_rmse[-1]:.4f}, Test: {test_rmse[-1]:.4f}')

        results = pd.DataFrame({
            'loss': loss,
            'train_rmse': train_rmse, 'val_rmse': val_rmse, 'test_rmse': test_rmse,
            'train_wrmse': train_wrmse, 'val_wrmse': val_wrmse, 'test_wrmse': test_wrmse,
            'time':(time.time()-t0)/60
        })
        
        ## Debugging
        #clear_output()
        '''
        print('\nloss: ', loss, 
              '\ntrain_rmse: ', train_rmse, 
              '\nval_rmse: ', val_rmse, 
              '\ntest_rmse: ', test_rmse,
              '\ntrain_wrmse: ', train_wrmse, 
              '\nval_wrmse: ', val_wrmse, 
              '\ntest_wrmse: ', test_wrmse,
              '\ntime: ', (time.time()-t0)/60)
        '''
        #visualize_loss(results, metric='wrmse').show()
        #print(results.to_string())

        # enable early stopping
        if (epoch > 1) and abs(loss[-1]/loss[-2]-1) < min_acc :
            k += 1
        if k> e_patience:
            print('Early stopping')
            break

    return results, model
"""

In [ ]:
'''
def visualize_loss(results, metric='rmse'):
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=results.index, y=results['train_'+metric], name = 'train_'+metric))
    fig.add_trace(go.Scatter(x=results.index, y=results['val_'+metric], name = 'val_'+metric))
    fig.add_trace(go.Scatter(x=results.index, y=results['test_'+metric], name = 'test_'+metric))
    fig.add_trace(go.Scatter(x=results.index, y=results['loss'], name = 'loss'))

    fig.update_yaxes(title_text=metric.upper())
    fig.update_xaxes(title_text="Epoch")

    return fig
'''

In [ ]:
'''
N_EPOCHS = 300
E_PATIENCE = 50
LEARNING_RATE = 0.01

model_params = {"hidden_channels":32, 'conv':SAGEConv}

results, trained_model = train_test(
    Model, model_params, learning_rate=LEARNING_RATE, e_patience = E_PATIENCE, n_epochs=N_EPOCHS)
'''

In [ ]:
#visualize_loss(results, metric='wrmse')

In [ ]:
#trained_model.encoder(test_data.x_dict, test_data.edge_index_dict)['session']

In [ ]:
#trained_model.state_dict()['encoder.linear2.session.weight']

# Recommendation

In [ ]:
'''
@torch.no_grad()
def recommendation(user_id, model, x_dict, edge_index_dict):
  # Get model decoder
  #model = Model(**model_params)
  with torch.no_grad():
    encoder = model.encoder(data.x_dict, data.edge_index_dict)

  # Get node representations for users and movies
  user_representations = encoder['session']
  movie_representations = encoder['aid']

  # Compute the dot product between user and movie representations to get edge weights
  edge_weights = user_representations.mm(movie_representations.T)

  # Make predictions for each user by taking the top k largest edge weights
  k = 20  # number of recommendations to make
  _, top_k_indices = edge_weights.topk(k, dim=1)
  recommendations = top_k_indices.numpy()

  # Print recommendations for the first user
  print(f'Recommendations for user {user_id}: {recommendations[user_id]}')
'''

In [ ]:
#session = 1
#recommendation(session, model, Rtest_data.x_dict, Rtest_data.edge_index_dict)

In [ ]:
#session = 2
#recommendation(session, model, data.x_dict, data.edge_index_dict)

In [ ]:
#session = 999
#recommendation(session, model, data.x_dict, data.edge_index_dict)

# Submission

Submission File
For each `session` id and `type` combination in the test set, you must predict the `aid` values in the `label` column, which is space delimited. You can predict up to 20 `aid` values per row. The file should contain a header and have the following format:

```
session_type,labels
12906577_clicks,135193 129431 119318 ...
12906577_carts,135193 129431 119318 ...
12906577_orders,135193 129431 119318 ...
12906578_clicks, 135193 129431 119318 ...
etc.
```